<a href="https://colab.research.google.com/github/Siddharth0317/osl/blob/main/Copy_of_Exp_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Import all required libraries

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import time
import copy
import os


In [ ]:
# Step 2: Define transformations for training and validation data

image_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}


In [ ]:
# Step 3: Load CIFAR-10 dataset automatically

train_data = datasets.CIFAR10(root='data', train=True, download=True,
                              transform=image_transforms['train'])
valid_data = datasets.CIFAR10(root='data', train=False, download=True,
                              transform=image_transforms['valid'])

batch_size = 32
dataloaders = {
    'train': DataLoader(train_data, batch_size=batch_size, shuffle=True),
    'valid': DataLoader(valid_data, batch_size=batch_size, shuffle=False)
}

class_names = train_data.classes
n_classes = len(class_names)

print("✅ CIFAR-10 loaded successfully!")
print("Classes:", class_names)

# For convenience in training loop
datasets_dict = {'train': train_data, 'valid': valid_data}


In [ ]:
# Step 4: Load the pre-trained VGG-16 model

from torchvision.models import vgg16, VGG16_Weights
model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
print(model)


In [ ]:
# Step 5: Freeze all convolutional layers

for param in model.features.parameters():
    param.requires_grad = False
print("✅ Convolutional layers frozen.")


In [ ]:
# Step 6: Add a custom classifier

n_inputs = model.classifier[6].in_features
model.classifier[6] = nn.Sequential(
    nn.Linear(n_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, n_classes),
    nn.LogSoftmax(dim=1)
)
print("✅ Custom classifier added:\n", model.classifier)


In [ ]:
# Step 7: Define loss function and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
print("✅ Loss and optimizer defined.")


In [ ]:
# Step 8: Train the model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

n_epochs = 3  # For demo; increase to 10–20 for better accuracy
train_loss_history, valid_loss_history = [], []

for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    print("-" * 20)

    for phase in ['train', 'valid']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(datasets_dict[phase])
        epoch_acc = running_corrects.double() / len(datasets_dict[phase])

        if phase == 'train':
            train_loss_history.append(epoch_loss)
        else:
            valid_loss_history.append(epoch_loss)

        print(f"{phase} Loss: {epoch_loss:.4f}  Acc: {epoch_acc:.4f}")

print("✅ Training completed.")


In [ ]:
# Step 9: Plot training vs validation loss

plt.figure(figsize=(8,6))
plt.plot(train_loss_history, label='Training Loss')
plt.plot(valid_loss_history, label='Validation Loss')
plt.title('Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Step 10: Save trained model

torch.save(model.state_dict(), 'transfer_learning_vgg16.pth')
print("✅ Model saved successfully as 'transfer_learning_vgg16.pth'!")
